# Self Attention

### 3.3 Attending to different parts of the input with self attention

#### 3.3.1 A simple self-attention mechanism without trainable weights

In [55]:
# consider that z^2 is the input vector x^2 but with information about all the other input
import torch

inputs = torch.tensor(
    [
        [0.43, 0.15, 0.89],  # Your     (x^1)
        [0.55, 0.87, 0.66],  # journey  (x^2)
        [0.57, 0.85, 0.64],  # starts   (x^3)
        [0.22, 0.58, 0.33],  # with     (x^4)
        [0.77, 0.25, 0.10],  # one      (x^5)
        [0.05, 0.80, 0.55],  # step     (x^6)
    ]
) # rows are words, columns are embedding dimensions
inputs

tensor([[0.4300, 0.1500, 0.8900],
        [0.5500, 0.8700, 0.6600],
        [0.5700, 0.8500, 0.6400],
        [0.2200, 0.5800, 0.3300],
        [0.7700, 0.2500, 0.1000],
        [0.0500, 0.8000, 0.5500]])

In [56]:
# Suppose we use the second input as query => q^2 = x^2
# The attention scores are: w21 = x^1 @ q^2.T; w22 = x^2 @ q^2.T; ... (w = omega)
# The attention weights are then these values normalized via softmax
query_2 = inputs[1]
score_dim = inputs.shape[0]
attn_scores_2 = torch.zeros(score_dim)
for i, input_i in enumerate(inputs):
    attn_scores_2[i] = torch.dot(input_i, query_2) # Multiplying each input with input_2 to get the attention scores for input_2
print(attn_scores_2)

tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])


In [57]:
query_2.shape

torch.Size([3])

In [58]:
# The above is equivalent to a matrix multiplication:
assert torch.equal(attn_scores_2, inputs @ query_2)

In [59]:
# Step 2: Normalize the scores so they sum up to 1 (they are thus called "weights")
# This is done to ease gradient learning, avoiding too large/small values
# Softmax does this best, by accounting for outliers
attn_weights_2 = torch.softmax(attn_scores_2, dim=0) # it's a just one row anyway 
attn_weights_2

tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])

In [60]:
print(inputs.shape)
print(attn_weights_2.shape)

torch.Size([6, 3])
torch.Size([6])


In [61]:
# Step 3: compute the context vector z_2 by multiplying the inputs by their respective weights
# Which is again just a matrix dot product
context_vec_2 = attn_weights_2 @ inputs
context_vec_2

tensor([0.4419, 0.6515, 0.5683])

#### 3.3.2 Computing attention weights for ALL input tokens

In [62]:
attn_scores_all = torch.empty(6,6)

# Apply previous Step 1 to all pairwise elements to compute the unnormalized attention score matrix
for i, x_i in enumerate(inputs):
    for j, x_j in enumerate(inputs):
        attn_scores_all[i, j] = x_i @ x_j

# Equivalent to:
# attn_scores_all = inputs @ inputs.T

print(attn_scores_all)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


In [63]:
print(inputs)
print()
print(inputs.T)

tensor([[0.4300, 0.1500, 0.8900],
        [0.5500, 0.8700, 0.6600],
        [0.5700, 0.8500, 0.6400],
        [0.2200, 0.5800, 0.3300],
        [0.7700, 0.2500, 0.1000],
        [0.0500, 0.8000, 0.5500]])

tensor([[0.4300, 0.5500, 0.5700, 0.2200, 0.7700, 0.0500],
        [0.1500, 0.8700, 0.8500, 0.5800, 0.2500, 0.8000],
        [0.8900, 0.6600, 0.6400, 0.3300, 0.1000, 0.5500]])


In [64]:
# step 2: normalize
attn_weights_all = torch.softmax(attn_scores_all, dim=1)
print(attn_weights_all)
attn_weights_all.sum(dim=-1)

tensor([[0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452],
        [0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581],
        [0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.1263, 0.1720],
        [0.1526, 0.1958, 0.1975, 0.1367, 0.1879, 0.1295],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])


tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000])

In [65]:
# Step 3: Get all context vectors z_i
context_vec_all = attn_weights_all @ inputs
print(context_vec_all)

tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])


These are now the same input tokens, modified to take context (surrounding words) into account.

# 3.4 Implementing self-attention with trainable weights (_fucked_)

A.k.a _scaled dot-product attention_

####  3.4.1 Implementing the weights step by step
We start by calculating the context vector for just one input element, x_2

In [66]:
x_2 = inputs[1]
d_in = inputs.shape[1] # input's embedding size, d=3
d_out = 2 # output's embedding size, d=2
# usually d_in==d_out, but we change this for better illustration

# Set the query, key, and value weight matrices
# The purpose of these weight matrices is to project each input into things that are trainable and usable in different scenarios (when input is considered the query, for example) 
torch.manual_seed(123)
W_query = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False) # This should be True, but we turn it off to reduce output clutter
W_key = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_value = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)

# Get q,k,v vecs
query_2 = x_2 @ W_query
key_2 = x_2 @ W_key
value_2 = x_2 @ W_value 

print(query_2) # 2d, since d_out=2

tensor([0.4306, 1.4551])


In [67]:
# Even though we're only trying to get z_2, we still need all keys and values (for the other inputs)
# because they're involved in computing the attention weights wrt q_2
keys = inputs @ W_key
values = inputs @ W_value
keys.shape

torch.Size([6, 2])

In [68]:
# Step 2: Compute the attention scores
# To do this, you multiply the given query by all the keys
attn_scores_2 = keys @ query_2 # same as doing query2 @ keys.T
print(attn_scores_2)

tensor([1.2705, 1.8524, 1.8111, 1.0795, 0.5577, 1.5440])


In [69]:
# Step 3: Getting attetion weights via scaled softmax
d_k = keys.shape[-1] # 2
attn_weights_2 = torch.softmax(attn_scores_2 / (d_k**0.5), dim=-1) # To prevent grads from getting too large and the softmax function from acting like a step function
print(attn_weights_2)

tensor([0.1500, 0.2264, 0.2199, 0.1311, 0.0906, 0.1820])


In [70]:
# Step 4: Context vec for x_2
context_vec_2 = attn_weights_2 @ values #* VALUES!
z_2 = context_vec_2
print(context_vec_2)

tensor([0.3061, 0.8210])


#### 3.4.2 Implementing a Python class for the self-attention module

In [71]:
import torch.nn as nn
from torch import softmax

class SelfAttention_v1(nn.Module):
    def __init__(self, d_in, d_out):
        super().__init__()
        self.W_query = nn.Parameter(torch.rand(d_in, d_out))
        self.W_key = nn.Parameter(torch.rand(d_in, d_out))
        self.W_value = nn.Parameter(torch.rand(d_in, d_out))

    def forward(self, x):
        queries = x @ W_query
        keys = x @ W_key
        values = x @ W_value
        attn_scores = queries @ keys.T # both are 6,2 with example input
        attn_weights = softmax(attn_scores / (keys.shape[-1]**0.5), dim=-1) # omega, or w
        context_vecs = attn_weights @ values
        return context_vecs
        

In [72]:
torch.manual_seed(123)
sa = SelfAttention_v1(d_in, d_out)
print(sa(inputs))

tensor([[0.2996, 0.8053],
        [0.3061, 0.8210],
        [0.3058, 0.8203],
        [0.2948, 0.7939],
        [0.2927, 0.7891],
        [0.2990, 0.8040]])


In [73]:
# Update the model by using Linear instead of Parameter, which
# has an optimized weight initialization scheme and, when bias=False,
# effectively does a matrix multiplication
class SelfAttention_v2(nn.Module):
    def __init__(self, d_in, d_out, kqv_bias=False):
        super().__init__()
        self.W_query = nn.Linear(d_in, d_out, bias=kqv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=kqv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=kqv_bias)

    def forward(self, x):
        queries = x @ W_query
        keys = x @ W_key
        values = x @ W_value
        attn_scores = queries @ keys.T  # both are 6,2 with example input
        attn_weights = softmax(attn_scores / (keys.shape[-1] ** 0.5), dim=-1)  # omega, or w
        context_vecs = attn_weights @ values
        return context_vecs

In [74]:
torch.manual_seed(789)
sa_v2 = SelfAttention_v2(d_in, d_out)
print(sa_v2(inputs))

tensor([[0.2996, 0.8053],
        [0.3061, 0.8210],
        [0.3058, 0.8203],
        [0.2948, 0.7939],
        [0.2927, 0.7891],
        [0.2990, 0.8040]])


### 3.5 Implementing causal/masked attention

#### 3.5.1 Causal attention mask

In [75]:
# Getting the weights so we can mask out the upper diagonal and normalize
queries = sa_v2.W_query(inputs)
keys = sa_v2.W_key(inputs)
attn_scores = queries @ keys.T
attn_weights = softmax(attn_scores / (keys.shape[-1]  ** 0.5), dim=-1)
print(attn_weights)

tensor([[0.1921, 0.1646, 0.1652, 0.1550, 0.1721, 0.1510],
        [0.2041, 0.1659, 0.1662, 0.1496, 0.1665, 0.1477],
        [0.2036, 0.1659, 0.1662, 0.1498, 0.1664, 0.1480],
        [0.1869, 0.1667, 0.1668, 0.1571, 0.1661, 0.1564],
        [0.1830, 0.1669, 0.1670, 0.1588, 0.1658, 0.1585],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<SoftmaxBackward0>)


In [76]:
# Creating a simple mask by setting the lower diagonal
diagonal = torch.tril(torch.ones_like(attn_weights))
masked_simple = attn_weights * diagonal
print(masked_simple)


tensor([[0.1921, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2041, 0.1659, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2036, 0.1659, 0.1662, 0.0000, 0.0000, 0.0000],
        [0.1869, 0.1667, 0.1668, 0.1571, 0.0000, 0.0000],
        [0.1830, 0.1669, 0.1670, 0.1588, 0.1658, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<MulBackward0>)


In [77]:
# Renormalize the weights using the row sums
row_sums = masked_simple.sum(dim=1, keepdim=True)
masked_simple_norm = masked_simple / row_sums
print(masked_simple_norm)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5517, 0.4483, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3800, 0.3097, 0.3103, 0.0000, 0.0000, 0.0000],
        [0.2758, 0.2460, 0.2462, 0.2319, 0.0000, 0.0000],
        [0.2175, 0.1983, 0.1984, 0.1888, 0.1971, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<DivBackward0>)


We can do this whole thing faster by inf'ing out the top diagonal of the attn_scores and taking the softmax of that

In [78]:
mask = torch.triu(torch.ones_like(attn_weights), diagonal=1)
masked_smart = attn_scores.masked_fill(mask.bool(), -torch.inf)
print(masked_smart)

tensor([[0.2899,   -inf,   -inf,   -inf,   -inf,   -inf],
        [0.4656, 0.1723,   -inf,   -inf,   -inf,   -inf],
        [0.4594, 0.1703, 0.1731,   -inf,   -inf,   -inf],
        [0.2642, 0.1024, 0.1036, 0.0186,   -inf,   -inf],
        [0.2183, 0.0874, 0.0882, 0.0177, 0.0786,   -inf],
        [0.3408, 0.1270, 0.1290, 0.0198, 0.1290, 0.0078]],
       grad_fn=<MaskedFillBackward0>)


In [79]:
attn_weights = softmax(masked_smart / keys.shape[-1]**0.5, dim=1)
print(attn_weights)
print(attn_weights.sum(dim=1))

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5517, 0.4483, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3800, 0.3097, 0.3103, 0.0000, 0.0000, 0.0000],
        [0.2758, 0.2460, 0.2462, 0.2319, 0.0000, 0.0000],
        [0.2175, 0.1983, 0.1984, 0.1888, 0.1971, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<SoftmaxBackward0>)
tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000],
       grad_fn=<SumBackward1>)


#### 3.5.2 Masking additional attention weights with dropout

In [80]:
torch.manual_seed(123)
dropout = torch.nn.Dropout(0.5) # drop half, p=.5
example = torch.ones(6,6)
dropout(example) # notice that the values are rescaled by 1/p. This is to ensure that average effect of attention stays the same.

tensor([[2., 2., 0., 2., 2., 0.],
        [0., 0., 0., 2., 0., 2.],
        [2., 2., 2., 2., 0., 2.],
        [0., 2., 2., 0., 0., 2.],
        [0., 2., 0., 2., 0., 2.],
        [0., 2., 2., 2., 2., 0.]])

#### 3.5.3 Implementing a compact causal attention class

In [81]:
# first, create a batched example so that it works with prev created class
batch = torch.stack((inputs, inputs))
print(batch.shape)
print()
print(inputs)
print()
print(batch)

torch.Size([2, 6, 3])

tensor([[0.4300, 0.1500, 0.8900],
        [0.5500, 0.8700, 0.6600],
        [0.5700, 0.8500, 0.6400],
        [0.2200, 0.5800, 0.3300],
        [0.7700, 0.2500, 0.1000],
        [0.0500, 0.8000, 0.5500]])

tensor([[[0.4300, 0.1500, 0.8900],
         [0.5500, 0.8700, 0.6600],
         [0.5700, 0.8500, 0.6400],
         [0.2200, 0.5800, 0.3300],
         [0.7700, 0.2500, 0.1000],
         [0.0500, 0.8000, 0.5500]],

        [[0.4300, 0.1500, 0.8900],
         [0.5500, 0.8700, 0.6600],
         [0.5700, 0.8500, 0.6400],
         [0.2200, 0.5800, 0.3300],
         [0.7700, 0.2500, 0.1000],
         [0.0500, 0.8000, 0.5500]]])


In [82]:
class CausalAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, qkv_bias=False):
        super().__init__()
        self.d_out = d_out
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key   = nn.Linear(d_in, d_out, bias=qkv_bias)         
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)         
        self.dropout = nn.Dropout(dropout) 
        self.register_buffer('mask', torch.triu(torch.ones(context_length, context_length), diagonal=1))
        
    def forward(self, x):
        b, num_tokens, d_in = x.shape
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)
        
        attn_scores = queries @ keys.transpose(1, 2) # switch dims 1 & 2
        attn_scores_masked = attn_scores.masked_fill_(self.mask.bool()[:num_tokens, :num_tokens], -torch.inf) # in-place cuz of "_" 
        attn_weights = softmax(attn_scores_masked / keys.shape[-1]**.5, dim=-1)
        attn_weights = self.dropout(attn_weights)
        context_vecs = attn_weights @ values
        return context_vecs

In [83]:
torch.manual_seed(123)
context_length = batch.shape[1]
ca = CausalAttention(d_in, d_out, context_length, 0.0)
context_vecs = ca(batch)
print(context_vecs)
print(context_vecs.shape)

tensor([[[-0.4519,  0.2216],
         [-0.5874,  0.0058],
         [-0.6300, -0.0632],
         [-0.5675, -0.0843],
         [-0.5526, -0.0981],
         [-0.5299, -0.1081]],

        [[-0.4519,  0.2216],
         [-0.5874,  0.0058],
         [-0.6300, -0.0632],
         [-0.5675, -0.0843],
         [-0.5526, -0.0981],
         [-0.5299, -0.1081]]], grad_fn=<UnsafeViewBackward0>)
torch.Size([2, 6, 2])


# 3.6 Extending single-head attention to multi-head attention

## 3.6.1 Stacking multiple single-head attention layers

In [84]:
# we start with a simple sequential version, and then we make it parallel
import torch.nn as nn


class MultiHeadAttentionWrapper(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        self.heads = nn.ModuleList(
            [
                CausalAttention(d_in, d_out, context_length, dropout, qkv_bias=qkv_bias)
                for _ in range(num_heads)
            ]
        )

    def forward(self, x):
        return torch.cat([head(x) for head in self.heads], dim=-1)


torch.manual_seed(123)
context_length = batch.shape[1]  # # of tokens
d_in, d_out = 3, 2
mha = MultiHeadAttentionWrapper(d_in, d_out, context_length, 0.0, 2)
context_vecs = mha(batch)

print(context_vecs.shape) # last dim=4 because d_out*2=2*2 (two heads). Remember that the two heads stack the vectors by column/vertically.
print(context_vecs)

torch.Size([2, 6, 4])
tensor([[[-0.4519,  0.2216,  0.4772,  0.1063],
         [-0.5874,  0.0058,  0.5891,  0.3257],
         [-0.6300, -0.0632,  0.6202,  0.3860],
         [-0.5675, -0.0843,  0.5478,  0.3589],
         [-0.5526, -0.0981,  0.5321,  0.3428],
         [-0.5299, -0.1081,  0.5077,  0.3493]],

        [[-0.4519,  0.2216,  0.4772,  0.1063],
         [-0.5874,  0.0058,  0.5891,  0.3257],
         [-0.6300, -0.0632,  0.6202,  0.3860],
         [-0.5675, -0.0843,  0.5478,  0.3589],
         [-0.5526, -0.0981,  0.5321,  0.3428],
         [-0.5299, -0.1081,  0.5077,  0.3493]]], grad_fn=<CatBackward0>)


## 3.6.2 Implementing multi-head attention with weight splits (parallel version)

In [85]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        assert (d_out % num_heads == 0), "d_out must be divisible by num_heads"
        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads # Reduce projection dim to match desired output dim

        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.out_proj = nn.Linear(d_out, d_out) # Linear layer to combine head outputs
        self.dropout = nn.Dropout(dropout)
        self.register_buffer("mask", torch.triu(torch.ones(context_length, context_length), diagonal=1))

    def forward(self, x):
        b, num_tokens, d_in = x.shape
        # As in `CausalAttention`, for inputs where `num_tokens` exceeds `context_length`,
        # this will result in errors in the mask creation further below.
        # In practice, this is not a problem since the LLM (chapters 4-7) ensures that inputs
        # do not exceed `context_length` before reaching this forward method.

        keys = self.W_key(x) # shape: (b, num_tokens, d_out)
        queries = self.W_query(x)
        values = self.W_value(x)

        # We implicitly split the matrix by adding a `num_heads` dimension
        # Unroll last dim: (b, num_tokens, d_out) -> (b, num_tokens, num_heads, head_dim)
        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim) # fine because loop-order stays the same, just unrolled
        values = values.view(b, num_tokens, self.num_heads, self.head_dim)
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)

        # Now transpose so that we have token-major order
        keys = keys.transpose(1, 2)
        queries = queries.transpose(1, 2)
        values = values.transpose(1, 2)

        # Compute scaled dot-product attention (self-attention) w/ causal mask
        attn_scores = queries @ keys.transpose(2, 3) # Dot product for each head

        # Original mask truncated to the number of tokens and converted to boolean
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]

        # Use the mask to fill attention scores
        attn_scores.masked_fill_(mask_bool, -torch.inf)

        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**.5, dim=-1)
        attn_weights = self.dropout(attn_weights)

        # shape: (b, num_tokens, num_heads, head_dim)
        context_vec = (attn_weights @ values).transpose(1, 2)

        # Combine heads, where self.d_out = self.num_heads * self.head_dim
        context_vec = context_vec.contiguous().view(b, num_tokens, self.d_out) # Need to call contiguous() because view expects data to be that way, and transpose changes it
        context_vec = self.out_proj(context_vec) # optional projection (used in gpt2 and most others)
        return context_vec


torch.manual_seed(123)

batch_size, context_length, d_in = batch.shape
d_out = 2
mha = MultiHeadAttention(d_in, d_out, context_length, 0.0, num_heads=2)

context_vecs = mha(batch)

print(context_vecs)
print("context_vecs.shape:", context_vecs.shape)

tensor([[[0.3190, 0.4858],
         [0.2943, 0.3897],
         [0.2856, 0.3593],
         [0.2693, 0.3873],
         [0.2639, 0.3928],
         [0.2575, 0.4028]],

        [[0.3190, 0.4858],
         [0.2943, 0.3897],
         [0.2856, 0.3593],
         [0.2693, 0.3873],
         [0.2639, 0.3928],
         [0.2575, 0.4028]]], grad_fn=<ViewBackward0>)
context_vecs.shape: torch.Size([2, 6, 2])


### Exercise 3.3
Using the MultiHeadAttention class, initialize a multi-head attention module that has the same number of attention heads as the smallest GPT-2 model (12 attention heads). Also ensure that you use the respective input and output embedding sizes similar to GPT-2 (768 dimensions). Note that the smallest GPT-2 model supports a context length of 1,024 tokens.

In [86]:
mha_big = MultiHeadAttention(768, 768, 1024, .5, 12)
print(mha)

MultiHeadAttention(
  (W_query): Linear(in_features=3, out_features=2, bias=False)
  (W_key): Linear(in_features=3, out_features=2, bias=False)
  (W_value): Linear(in_features=3, out_features=2, bias=False)
  (out_proj): Linear(in_features=2, out_features=2, bias=True)
  (dropout): Dropout(p=0.0, inplace=False)
)


In [87]:
inputs = torch.rand(768, 768)
print(inputs.shape)
batch = torch.stack((inputs, inputs))
print(batch.shape)

torch.Size([768, 768])
torch.Size([2, 768, 768])


In [91]:
# cpu
%timeit mha_big(batch)

# gpu
batch_gpu = batch.to("cuda")
mha_big_gpu = MultiHeadAttention(768, 768, 1024, .5, 12).to("cuda")
%timeit mha_big_gpu(batch_gpu)

126 ms ± 3.82 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
542 μs ± 142 μs per loop (mean ± std. dev. of 7 runs, 1 loop each)
